In [ ]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Input
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Rescaling, GlobalAveragePooling2D, Input, Flatten, RandomBrightness, RandomContrast, Dropout, MaxPooling2D, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
IMG_SIZE = 180

In [ ]:
datagen_train = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)


In [ ]:
train_set = datagen_train.flow_from_directory("../dataset/ttte/train", batch_size=8, target_size=(IMG_SIZE, IMG_SIZE), class_mode='categorical')

In [ ]:
test_set = datagen_test.flow_from_directory("../dataset/ttte/validation", batch_size=8, target_size=(IMG_SIZE, IMG_SIZE), class_mode='categorical')

In [ ]:
train_set.class_indices

In [ ]:
test_set.class_indices

In [ ]:
STEP_SIZE_TRAIN = train_set.samples // train_set.batch_size
STEP_SIZE_TRAIN

In [ ]:
STEP_SIZE_TEST = test_set.samples // test_set.batch_size
STEP_SIZE_TEST

In [ ]:
data_augmentation = Sequential(
    [

    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1)
    ]
)
data_augmentation

In [ ]:
input_shape = IMG_SIZE, IMG_SIZE, 3
base_model = MobileNetV2(
    input_shape=input_shape,
    include_top=False,  
    weights='imagenet'
)
base_model.trainable = True

base_model.summary()

In [ ]:
for layer in base_model.layers[:-50]:
    layer.trainable = False

In [ ]:
model = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    data_augmentation,
    # Rescaling(1./255),
    base_model,
    Flatten(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(len(train_set.class_indices), activation='softmax')
    # Conv2D(16,(3,3), padding='same', activation = 'relu'),
    # MaxPooling2D(padding='same'),
    # Conv2D(32,(3,3), padding = 'same', activation = 'relu'),
    # MaxPooling2D(padding='same'),
    # Conv2D(64,(3,3), padding = 'same', activation = 'relu'),
    # MaxPooling2D(padding='same'),
    # Dropout(0.3),
    # Flatten(),
    # Dense(256, activation = 'relu'),
    # Dropout(0.4),
    # Dense(len(train_set.class_indices), activation='softmax')
])
model.summary()


In [ ]:
# for layer in base_model.layers[:-30]:
#     layer.trainable = False

In [ ]:
opt = Adam(learning_rate=1e-4)

callback = EarlyStopping(monitor='val_accuracy',
                         patience=5,
                         verbose=1,
                         mode="max",
                         baseline=0.5,
                          restore_best_weights=True,
                          start_from_epoch=5
                         )


In [ ]:

model.compile(
            optimizer=opt,
              loss = CategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])

history = model.fit(train_set,
                           epochs=15,
                           steps_per_epoch=STEP_SIZE_TRAIN,
                           validation_data=test_set, 
                            validation_steps=STEP_SIZE_TEST,
                           verbose=1,
                           callbacks=[callback]
                           )


In [ ]:
model.save("ttte_model.keras")